**{DEV}\[TODO\]**
- everything
- QUERY: n-link or double?

---
### Chapter 1: System Modelling
---
# **Tutorial 1.d: Relative vs Absolute Orientations in 3D**
**Aim:** To compare the performance of simulation when using relative vs absolute orientations in three dimensions

We've already established how to set up the modelling code, and compared relative vs absolute orientations in 2D. Now we will do the same in 3D

### **Contents**:
* [3D Modelling](#3D-Modelling)
* [Spherical Pendulum](#Spherical-Pendulum)
* [Hinge Pendulum](#Hinge-Pendulum)

## **3D Modelling**

In [25]:
# import libraries
import sympy as sym
import numpy as np
import time # for benchmarking different models

from IPython.display import display, HTML #for pretty printing
display(HTML("<style>.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea { max-height: 32em; }</style>"))

# define time-logging function for benchmarking
def log_time(t_start=None, msg=""):
    t_end = time.perf_counter()
    if t_start == None:
        if msg == "": print("Timer initialized")
        else: print(msg)
    else: print(msg + f": {t_end - t_start:.6f} sec")
    return t_end

# create symbolic variables - Generalized for n-link
nlinks = 2
links = np.arange(nlinks)

# --------------------------------------------------------------------------------------------------------------

# system parameters
g = sym.symbols('g')

m  = [sym.symbols( 'm_{%s}' %(i+1)) for i in links] # mass of links
l  = [sym.symbols( 'l_{%s}' %(i+1)) for i in links] # length of links

I_psi = [sym.symbols('I_{\\psi_{%s}}'   %(i+1)) for i in links] # moment of intertia of links
I_th  = [sym.symbols('I_{\\theta_{%s}}' %(i+1)) for i in links]
I_phi = [sym.symbols('I_{\\phi_{%s}}'   %(i+1)) for i in links]

# generalized coordinates
X0, Y0, Z0 = sym.symbols(['X_{0}','Y_{0}', 'Z_{0}']) # fixed position of first link

psi   = [sym.symbols(       '\\psi_{%s}' %(i+1)) for i in links] #positions
dpsi  = [sym.symbols( '\dot{\\psi}_{%s}' %(i+1)) for i in links] #velocities
ddpsi = [sym.symbols('\ddot{\\psi}_{%s}' %(i+1)) for i in links] #accelerations

th   = [sym.symbols(       '\\theta_{%s}' %(i+1)) for i in links] #positions
dth  = [sym.symbols( '\dot{\\theta}_{%s}' %(i+1)) for i in links] #velocities
ddth = [sym.symbols('\ddot{\\theta}_{%s}' %(i+1)) for i in links] #accelerations

phi   = [sym.symbols(       '\\phi_{%s}' %(i+1)) for i in links] #positions
dphi  = [sym.symbols( '\dot{\\phi}_{%s}' %(i+1)) for i in links] #velocities
ddphi = [sym.symbols('\ddot{\\phi}_{%s}' %(i+1)) for i in links] #accelerations

q   = sym.Matrix([[  psi[i],  th[i],  phi[i]] for i in links])
dq  = sym.Matrix([[ dpsi[i], dth[i], dphi[i]] for i in links])
ddq = sym.Matrix([[ddpsi[i],ddth[i],ddphi[i]] for i in links])

q   =   q.reshape(q.rows * q.cols, 1)
dq  =  dq.reshape(q.rows * q.cols, 1)
ddq = ddq.reshape(q.rows * q.cols, 1)


# --------------------------------------------------------------------------------------------------------------

# STEP 1: system space coordinates written in terms of the generalised coordinates

# helper functions
# the 3D system space coordinates are [x;y;z;psi;th;phi], so we need 3 rotation functions that can work with this
def MatAugment(R1, R2):
    # takes in a 3D rotation matrix and augments it to form a 6x6 matrix
    empty_col = sym.Matrix([0,0,0])
    R = R1.col_insert(3, empty_col)
    R = R1.col_insert(4, empty_col)
    R = R1.col_insert(5, empty_col)
    R = R1.row_insert(3, sym.Matrix([0,0,0,R2[0,0],R2[0,1],R2[0,2]]).T)
    R = R1.row_insert(4, sym.Matrix([0,0,0,R2[1,0],R2[1,1],R2[1,2]]).T)
    R = R1.row_insert(5, sym.Matrix([0,0,0,R2[2,0],R2[2,1],R2[2,2]]).T)
    return R

def Rotate(v, psi, th, phi, order):
    Rx = sym.Matrix([[1,            0,             0],
                     [0, sym.cos(phi), -sym.sin(phi)],
                     [0, sym.sin(phi),  sym.cos(phi)]])
    
    Ry = sym.Matrix([[  sym.cos(th), 0,  sym.sin(th)],
                     [            0, 1,            0],
                     [ -sym.sin(th), 0,  sym.cos(th)]])
    
    Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                     [sym.sin(psi),  sym.cos(psi), 0],
                     [           0,             0, 1]])
    
#     Rx = MatAugment(Rx)
#     Ry = MatAugment(Ry)
#     Rz = MatAugment(Rz)
    
    S = sym.Matrix([[0],[0],[0]])
    return R*v + S

In [17]:
def MatAugment(R1, R2):
    # takes in a 3D rotation matrix and augments it to form a 6x6 matrix
    empty_col = sym.Matrix([0,0,0])
    R = R1.col_insert(3, empty_col)
    R = R.col_insert(4, empty_col)
    R = R.col_insert(5, empty_col)
    R = R.row_insert(3, sym.Matrix([0,0,0,R2[0,0],R2[0,1],R2[0,2]]).T)
    R = R.row_insert(4, sym.Matrix([0,0,0,R2[1,0],R2[1,1],R2[1,2]]).T)
    R = R.row_insert(5, sym.Matrix([0,0,0,R2[2,0],R2[2,1],R2[2,2]]).T)
    return R

Rx = sym.Matrix([[1,               0,                0],
                 [0, sym.cos(phi[0]), -sym.sin(phi[0])],
                 [0, sym.sin(phi[0]),  sym.cos(phi[0])]])

Ry = sym.Matrix([[  sym.cos(th[0]), 0,  sym.sin(th[0])],
                 [               0, 1,            0],
                 [ -sym.sin(th[0]), 0,  sym.cos(th[0])]])

Rz = sym.Matrix([[sym.cos(psi[0]), -sym.sin(psi[0]), 0],
                 [sym.sin(psi[0]),  sym.cos(psi[0]), 0],
                 [              0,                0, 1]])

# display(MatAugment(Rx,Rz))

**BELOW WON'T WORK. Why? Rotations do not simply add on like in 2D**

In [ ]:
# helper functions
# the 3D system space coordinates are [x;y;z;psi;th;phi], so we need 3 rotation functions that can work with this
def MatAugment(R):
    # takes in a 3D rotation matrix and augments it to form a 6x6 matrix
    empty_col = sym.Matrix([0,0,0])
    R = R.col_insert(3, empty_col)
    R = R.col_insert(4, empty_col)
    R = R.col_insert(5, empty_col)
    R = R.row_insert(3, sym.Matrix([0,0,0,1,0,0]).T)
    R = R.row_insert(4, sym.Matrix([0,0,0,0,1,0]).T)
    R = R.row_insert(5, sym.Matrix([0,0,0,0,0,1]).T)
    return R
def RotX(v, phi):
    R = sym.Matrix([[1,            0,             0],
                    [0, sym.cos(phi), -sym.sin(phi)],
                    [0, sym.sin(phi),  sym.cos(phi)]])
    R = MatAugment(R)
    S = sym.Matrix([[0],[0],[0],[0],[0],[phi]])
    return R*v + S
def RotY(v, th):
    R = sym.Matrix([[  sym.cos(th), 0,  sym.sin(th)],
                    [            0, 1,            0],
                    [ -sym.sin(th), 0,  sym.cos(th)]])
    R = MatAugment(R)
    S = sym.Matrix([[0],[0],[0],[0],[th],[0]])
    return R*v + S
def RotZ(v, psi):
    R = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                    [sym.sin(psi),  sym.cos(psi), 0],
                    [           0,             0, 1]])
    R = MatAugment(R)
    S = sym.Matrix([[0],[0],[0],[psi],[0],[0]])
    return R*v + S

## **Spherical Pendulum**

## **Hinge Pendulum**